In [1]:
import pandas as pd
# Use a pipeline as a high-level helper
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from tqdm import tqdm
from openai import OpenAI
import os
from evaluate import evaluate_models

/Users/jasonchung/Downloads/nlp/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
api_key = os.getenv("DEEPSEEK_API_KEY")
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

In [51]:
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct", max_new_tokens=2048)
pipe(messages)

Device set to use mps:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'}]}]

In [45]:
def call_model(content):
    messages = [{"role": "system", "content": 
                 "You are an AI assistant who will assess whether a given citation text is used as a background, result or method section in a scientific paper."
                 "You will be given a json object containing some metadata, where the field \"string\" contains the citation text, "
                 "and you will need to classify the text as one of [background, result, method]."
                 "You will also need to provide a reasoning for your classification."
                 "Example output would look like: {\"classification\": \"result\", \"reasoning\": \"This is the reasoning\"}"
                 "You will be given 10 of such json objects to classify, and you are strictly required to return all 10 classifications with reasonings, separated by a new line."
                 "There should not be any extra new line characters within each of the json classification"
                 
    }, 
    {"role": "user", "content": f"{content}"}]
    response = client.chat.completions.create( model="deepseek-reasoner", messages=messages)
    print(response)
    try:
        if response.choices[0].message.content:
            #print(response["generated_text"][2]["content"])
            content = response.choices[0].message.content.split("\n")
            result = []
            for res in content:
                print(res)
                start = res.find("{")
                end = res.rfind("}") + 1
                json_str = res[start:end]
                answers = json.loads(json_str)
                classification = answers["classification"]
                reasoning = answers["reasoning"]
                result.append([classification, reasoning])
            return result
        else:
            return "Invalid", "No reasoning provided"
    except Exception as e:
        print(e)
        return "Invalid", "No reasoning provided"

In [4]:

def preprocess(file_path):
    data = pd.read_json(path_or_buf=file_path, lines=True)
    ids = set()
    rows_to_be_dropped = []
    for i in range(len(data)):
        row = data.iloc[i]
        if row.unique_id in ids:
            rows_to_be_dropped.append(i)
        else:
            ids.add(row.unique_id)
    data = data.drop(rows_to_be_dropped)
    print(set(data["label2"]))
    data = data.drop(columns=["label", "label_confidence", "label2_confidence", "label2"])
    return data

In [5]:
data = preprocess("./data/train.jsonl")

{'supportive', 'cant_determine', nan, 'not_supportive'}


In [41]:
first_partition = data[:1365]
second_partition = data[1365:2730]
third_partition = data[2730:4095]
fourth_partition = data[4095:5460]
fifth_partition = data[5460:6825]
sixth_partition = data[6825:]

In [ ]:
labels = []
reasonings = []
raw_output = []
ids = []

for i in range(0, len(first_partition), 10):
    if i % 50 == 0:
        print(i)
    current_data = [first_partition.iloc[i + x].to_json() for x in range(min(10, len(first_partition) - i))]
    for x in range(min(10, len(first_partition) - i)):
        ids.append(first_partition.iloc[i + x].unique_id)
    results = call_model(current_data)
    for result in results:
        labels.append(result[0])
        reasonings.append(result[1])

0


In [ ]:
for i in range(0, len(fourth_partition), 10):
    if i % 50 == 0:
        print(i)
    current_data = [fourth_partition.iloc[i + x].to_json() for x in range(min(10, len(fourth_partition) - i))]
    for x in range(min(10, len(fourth_partition) - i)):
        ids.append(fourth_partition.iloc[i + x].unique_id)
    results = call_model(current_data)
    for result in results:
        labels.append(result[0])
        reasonings.append(result[1])

700
ChatCompletion(id='6ae0d201-0bd5-4811-a57c-74070fab5283', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"classification": "method", "reasoning": "The citation refers to a previous study\'s finding used to establish a threshold in the current methodology, situated within the Methods section."}\n{"classification": "result", "reasoning": "The citation compares current findings with prior results on SHP-2 binding, located in the Results and Discussion section."}\n{"classification": "method", "reasoning": "Describes a methodological choice (using the VEGA package) to calculate SASA values, explicitly in the Methods section."}\n{"classification": "result", "reasoning": "Cites a prior experimental finding about Aβ ratio changes, supporting mechanistic discussion in a results-focused section."}\n{"classification": "result", "reasoning": "Summarizes advantages of the flipped classroom approach by citing prior studies\' conclusions, pre

IndexError: single positional indexer is out-of-bounds

In [ ]:
df = pd.DataFrame(zip(ids, labels, reasonings), columns=["id", "model_classification", "reasoning"])
df.to_csv("deepseek_third.csv")